# Demo for Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Import data

In [ ]:
# Data for Machine Learning is generally stored as csv files.
# You can think of them as excel files with rows and columns
df = pd.read_csv('UniLinRegData.csv')
df.head(10)

You can plot the data using matplotlib to get a nice graphical view of the problem.

In [ ]:
plt.figure(figsize=(15, 10))    # making the plot bigger
plt.scatter(df['x1'], df['y'])
plt.show()

This data looks a bit too good! That is because we generated it synthetically (using numpy!)

# Linear Regression

If a column of 1s is added to our data as x0, we will have an easier time dealing as vectors!

In [ ]:
df.insert(1, "x0", np.ones(df.shape[0]), True)
df.head()

We need to split the data to training and testing 

In [ ]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(df[['x1', 'x0']], df['y'], test_size=0.2, random_state=42)

In [ ]:
X_Train = X_Train.to_numpy()
X_Test = X_Test.to_numpy()
Y_Train = Y_Train.to_numpy().reshape(-1, 1)
Y_Test = Y_Test.to_numpy().reshape(-1, 1)

Note that now we have each <b>Datapoint</b> as a vector: $$x_{i} = \begin{bmatrix} x1 \\ x0 \end{bmatrix}$$

And, our dataset, Matrix X is: $$X = \begin{bmatrix} x1_{1}&x0_{1}\\x1_{2}&x0_{2}\\.\\.\\x1_{n}&x0_{n} \end{bmatrix}$$

Or $$X = \begin{bmatrix} x_{1}^T\\x_{2}^T\\.\\.\\x_{n}^T \end{bmatrix}$$

And $$y = \begin{bmatrix} y_{1} \\ y_{2} \\.\\. \\y_{n} \end{bmatrix}$$

As always, we will make a class for our model. Inside it, we can easily declare methods and attributes to help train the data.

In the class we have different methods like loss, gradient, etc. With what you have seen, try to write some code in numpy for the same! Take care of the dimensions (or the shape) of your numpy array!

Here are the formulae for Linear Regression as discussed:
$$\text{Closed Form: } \vec{\theta} = (X^TX)^{-1}X^TY$$
$$\text{Loss: } J(\vec{\theta}) = (\vec{y} - X\vec{\theta})^T(\vec{y} - X\vec{\theta})$$
$$\text{Gradient: } \frac{\partial J}{\partial \vec{\theta}} = 2(X^TX\vec{\theta} - X^T\vec{y})$$
$$\text{Gradient Descent Update: } \vec{\theta_{new}} := \vec{\theta_{old}} - \alpha \frac{\partial J}{\partial \vec{\theta}}$$

Some helpful numpy methods you can use:
- Use .T after any vector/matrix to get its transpose
- np.matmaul: for matrix multiplications and dot products
- np.linalg.inv: to find matrix inverses

In [ ]:
class LinReg:
    def __init__(self, dimensions):
        self.theta_grad = np.random.rand(dimensions, 1)
        self.theta_closed = np.random.rand(dimensions, 1)

    def closed_form(self, X, Y):
        self.theta_closed = np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, Y))
        # return self.theta_closed
        # pass

    def loss(self, X, Y):
        Loss = np.matmul((Y - np.matmul(X, self.theta_grad)).T, (Y - np.matmul(X, self.theta_grad)))

        return np.squeeze(Loss)

    def gradient(self, X, Y):
        # dJ(w)/dw = 2*(XT*X*W - XT*Y)
        gradient = 2 * (np.matmul(np.matmul(X.T, X), self.theta_grad) - np.matmul(X.T, Y))
        return gradient
        # pass

    def update(self, X, Y, alpha):
        # wnew = wold - alpha * dJ(w)/dw

        self.theta_grad = self.theta_grad - alpha * self.gradient(X, Y)
        # pass     

    


### Initialise the Model

We need to initialise the weights randomly, this step is done as the Model object is made. We need to pass in the number of features (columns) in X so that weights are initiated accordingly

In [ ]:
LinRegModel = LinReg(X_Train.shape[1])

#### Here we find our closed form solution

In [ ]:
LinRegModel.closed_form(X_Train, Y_Train);

In [ ]:
start = 50
stop = 500
multiplier = 3
linX_CF = np.linspace(start, stop, multiplier*stop-start)
linY_CF = linX_CF * LinRegModel.theta_closed[0] + LinRegModel.theta_closed[1]

In [ ]:

plt.figure(figsize=(15, 10))
plt.plot(linX_CF, linY_CF, '-r')
plt.scatter(X_Train[:, 0], Y_Train)
plt.show()

In [ ]:

plt.figure(figsize=(15, 10))
plt.plot(linX_CF, linY_CF, '-r')
plt.scatter(X_Test[:, 0], Y_Test)
plt.show()

#### Here we find our Gradient Descent Solution

In [ ]:
no_of_epochs = 50   # These are the number of repetitions we require in gradient descent
alpha = 0.000000005

for i in range(no_of_epochs):
    Loss = LinRegModel.loss(X_Train, Y_Train)
    if i % 5 == 0: print("Loss =", Loss)
    LinRegModel.update(X_Train, Y_Train, alpha)

Let us see how the line fits our data!

In [ ]:
start = 50
stop = 500
multiplier = 3
linX_Gr = np.linspace(start, stop, multiplier*stop-start)
linY_Gr = linX_Gr * LinRegModel.theta_grad[0] + LinRegModel.theta_grad[1]

In [ ]:

plt.figure(figsize=(15, 10))
plt.plot(linX_Gr, linY_Gr, '-r')
plt.scatter(X_Train[:, 0], Y_Train)
plt.show()

In [ ]:

plt.figure(figsize=(15, 10))
plt.plot(linX_Gr, linY_Gr, '-r')
plt.scatter(X_Test[:, 0], Y_Test)
plt.show()

### Errors for Closed Form

We need to write the predictions according to the X we use

In [ ]:
Pred_Train_CF = np.matmul(X_Train, LinRegModel.theta_closed)
Pred_Test_CF = np.matmul(X_Test, LinRegModel.theta_closed)

In [ ]:
print("MSE for Training =", mean_squared_error(Y_Train, Pred_Train_CF))
print("MSE for Testing  =", mean_squared_error(Y_Test, Pred_Test_CF))

### Errors for Gradient Descent

We need to write the predictions according to the X we use

In [ ]:
Pred_Train_Gr = np.matmul(X_Train, LinRegModel.theta_grad)
Pred_Test_Gr = np.matmul(X_Test, LinRegModel.theta_grad)

In [ ]:
print("MSE for Training =", mean_squared_error(Y_Train, Pred_Train_Gr))
print("MSE for Testing  =", mean_squared_error(Y_Test, Pred_Test_Gr))

## Let us put this against SciKit-Learn!

In [ ]:
LinRegSKL = LinearRegression()

In [ ]:
LinRegSKL.fit(X_Train, Y_Train);

In [ ]:
theta_SKL = np.array([LinRegSKL.coef_[0, 0], LinRegSKL.intercept_[0]]).reshape(-1, 1)

In [ ]:
Pred_Train_SKL = np.matmul(X_Train, theta_SKL)
Pred_Test_SKL = np.matmul(X_Test, theta_SKL)

In [ ]:
print("MSE for Training =", mean_squared_error(Y_Train, Pred_Train_SKL))
print("MSE for Testing  =", mean_squared_error(Y_Test, Pred_Test_SKL))